In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
def read_images(dataset_path):
    global classes
    im_path,labels=list(),list()
    classes=os.listdir(dataset_path)
    for i in classes:
        path=os.path.join(dataset_path,i)
        for j in os.listdir(path):
            path1=os.path.join(path,j)
            im_path.append(path1)
            lab=classes.index(i)
            if lab==0:
                labels.append([0,1])
            else:
                labels.append([1,0])
    for i in range(len(classes)):
        print('{}:{}'.format(classes[i],i))
    im_path=tf.convert_to_tensor(im_path)
    labels=tf.convert_to_tensor(labels)
    labels=tf.cast(labels,tf.float32)
    image,labels=tf.train.slice_input_producer([im_path,labels],shuffle=True)
    image = tf.read_file(image)
    image = tf.image.decode_jpeg(image, channels=1)
    image = tf.image.resize_images(image, [32, 32])
    # Normalize
    image = image * 1.0/127.5 - 1.0
    X, Y = tf.train.batch([image, labels], batch_size=32,capacity=32 * 8,num_threads=4)
    return X,Y

In [3]:
# dataset_path=''
dataset_path='dataset/train/'
X,Y=read_images(dataset_path)

stone:0
paper:1
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions f

In [4]:
def conv2d(x,input,output,filter=3,strides=1,padding='SAME',activation='relu',name=None):
    w=tf.Variable(tf.random_normal([filter,filter,input,output]))
    b=tf.Variable(tf.random_normal([output]))
    x=tf.nn.conv2d(x,w,strides=[1,strides,strides,1],padding=padding)
    x=tf.nn.bias_add(x,b)
    if activation is 'relu':
        x=tf.nn.relu(x,name=name)
    elif(activation is 'tanh'):
        x=tf.nn.tanh(x,name=name)
    elif(activation is 'softmax'):
        x=tf.nn.softmax(x,name=name)
    else:
        print('give relu,tanh or softmax')
    return x


def maxpool2d(x, k=2,name=None):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME',name=name)

def averagepool2d(x,k=2,name=None):
    return tf.nn.avg_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME',name=name)

def dense(x,input,output,activation='tanh',name=None):
    w=tf.Variable(tf.random_normal([input,output]))
    print(w)
    b=tf.Variable(tf.random_normal([output]))
    
    x=tf.add(tf.matmul(x,w),b)
    
    if(activation is 'tanh'):
        x=tf.nn.tanh(x,name=name)
    elif(activation is 'softmax'):
        x=tf.nn.softmax(x,name=name)
    elif(activation is 'relu'):
        x=tf.nn.relu(x,name=name)
    else:
        print('give tanh or softmax')
    return x
    

def lenet(x,re):
    with tf.compat.v1.variable_scope('LeNet',reuse=re):
        conv1=conv2d(x,1,6,filter=5,activation='relu',padding='VALID',name='Conv1')
        print conv1.name[:-6],conv1.shape
        conv1=averagepool2d(conv1,name='Avgpool1')
        print conv1.name[:-6],conv1.shape
        conv2=conv2d(conv1,6,16,filter=5,activation='relu',padding='VALID',name='Conv2')
        print conv2.name[:-6],conv2.shape
        conv2=averagepool2d(conv2,name='Avgpool1')
        flatten=tf.reshape(conv2,[conv2.get_shape()[0],conv2.get_shape()[1]*conv2.get_shape()[2]*
                                  conv2.get_shape()[3]],name='flatten')
        print conv2.name[:-6],conv2.shape
        print flatten.name[:-6],flatten.shape


        d1=dense(flatten,input=flatten.get_shape().as_list()[1],output=120,activation='relu',name='Dense1')
        print d1.name[:-6],d1.shape
        d2=dense(d1,input=120,output=84,activation='relu',name='Dense2')
        print d2.name[:-6],d2.shape
        model=dense(d2,input=84,output=2,activation='softmax',name='Dense3')
        print model.name[:-6],model.shape
    return model

In [5]:
x=tf.placeholder(tf.float32,[None,32,32,1])

In [6]:
model_tr=lenet(X,re=False)
# tf.compat.v1.variable_scope('Lenet',reuse=True)
model_te=lenet(X,re=True)

Instructions for updating:
Colocations handled automatically by placer.
LeNet/C (32, 28, 28, 6)
LeNet/Avgp (32, 14, 14, 6)
LeNet/C (32, 10, 10, 16)
LeNet/Avgpoo (32, 5, 5, 16)
LeNet/fla (32, 400)
<tf.Variable 'LeNet/Variable_4:0' shape=(400, 120) dtype=float32_ref>
LeNet/De (32, 120)
<tf.Variable 'LeNet/Variable_6:0' shape=(120, 84) dtype=float32_ref>
LeNet/De (32, 84)
<tf.Variable 'LeNet/Variable_8:0' shape=(84, 2) dtype=float32_ref>
LeNet/De (32, 2)
LeNet_1/C (32, 28, 28, 6)
LeNet_1/Avgp (32, 14, 14, 6)
LeNet_1/C (32, 10, 10, 16)
LeNet_1/Avgpoo (32, 5, 5, 16)
LeNet_1/fla (32, 400)
<tf.Variable 'LeNet_1/Variable_4:0' shape=(400, 120) dtype=float32_ref>
LeNet_1/De (32, 120)
<tf.Variable 'LeNet_1/Variable_6:0' shape=(120, 84) dtype=float32_ref>
LeNet_1/De (32, 84)
<tf.Variable 'LeNet_1/Variable_8:0' shape=(84, 2) dtype=float32_ref>
LeNet_1/De (32, 2)


In [7]:
pred=tf.nn.softmax(model_tr)

In [8]:
val=tf.nn.softmax_cross_entropy_with_logits(logits=model_tr,labels=Y)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
loss_op=tf.reduce_mean(val)

In [10]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.001)

In [11]:
train_op=optimizer.minimize(loss_op)

In [12]:
a=tf.argmax(model_te,1)
b=tf.argmax(tf.cast(Y,tf.int64),1)
correct_pred=tf.equal(a,b)

In [13]:
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [14]:
epoch=1000
init=tf.global_variables_initializer()
with tf.Session() as sess:
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(coord=coord)
    print(len(threads))
    sess.run(init)
    for j in range(epoch):
        for i in range(0,6466//32):
            sess.run(train_op)
#             print(sess.run(model_tr))
#             print(' ')
#             print(sess.run(Y))
#             print('<<<<<<<>>>>>>>>>>>>')
        print(sess.run(loss_op))
        for k in tf.global_variables():
            if k.name in ['LeNet/Variable_4:0','LeNet/Variable_6:0','LeNet/Variable_8:0']:
#                 print(sess.run(k))
                break
        acc=sess.run(accuracy)    
        loss=sess.run(loss_op)
        print('Epoch:{}/{} Accuracy:{:.3f} Loss:{:.3f}'.format(j+1,epoch,acc,loss))
        if j == 10:
            break
    coord.request_stop()
    coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
7
0.6882616
Epoch:1/1000 Accuracy:0.281 Loss:0.876
0.87576157
Epoch:2/1000 Accuracy:0.219 Loss:0.813
0.8445116
Epoch:3/1000 Accuracy:0.344 Loss:1.001
0.78201157
Epoch:4/1000 Accuracy:0.531 Loss:0.751
1.0320116
Epoch:5/1000 Accuracy:0.156 Loss:0.813
0.8757616
Epoch:6/1000 Accuracy:0.625 Loss:0.813
0.71951157
Epoch:7/1000 Accuracy:0.406 Loss:0.876
0.9070116
Epoch:8/1000 Accuracy:0.375 Loss:0.688
0.6882616
Epoch:9/1000 Accuracy:0.375 Loss:0.782
0.7195116
Epoch:10/1000 Accuracy:0.375 Loss:1.001
0.8445116
Epoch:11/1000 Accuracy:0.438 Loss:0.751


In [15]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in tf.global_variables():
        if i.name in ['LeNet/Variable_4:0','LeNet/Variable_6:0','LeNet/Variable_8:0']:
            print(sess.run(i))
            break

[[ 0.6622913  -1.4116085   1.4512349  ... -0.16379796 -0.781739
   0.37344053]
 [-0.6719981   0.28586602 -0.42753226 ... -0.33789095  0.9940902
  -1.1872883 ]
 [ 0.12809117  0.9538659   1.9474157  ...  0.22666968 -1.3512485
  -2.1260288 ]
 ...
 [-0.6067873   1.0030605   0.239813   ...  1.088203    1.9643157
   0.6042882 ]
 [-1.8572958   0.923164   -0.84635067 ...  1.1361066   0.7730969
   0.29701898]
 [ 0.80195594 -0.47840452  0.40852252 ...  1.5774734   0.17316936
   0.6252671 ]]


#### Reference : https://github.com/aymericdamien/TensorFlow-Examples